# Step 1 training model

In [2]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [22]:
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB2, EfficientNetB3, EfficientNetB4, EfficientNetB5, EfficientNetB6
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [4]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    print("Device:", tpu.master())
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    print("Not connected to a TPU runtime. Using CPU/GPU strategy")
    strategy = tf.distribute.MirroredStrategy()

Not connected to a TPU runtime. Using CPU/GPU strategy


In [5]:
IMG_SIZE = 224

batch_size = 64

dataset_name = "stanford_dogs"

(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

Generating train examples...:   0%|          | 0/12000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/stanford_dogs/0.2.0.incomplete4KPLX2/stanford_dogs-train.tfrecord*...:   0…

Generating test examples...:   0%|          | 0/8580 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/stanford_dogs/0.2.0.incomplete4KPLX2/stanford_dogs-test.tfrecord*...:   0%…

Dataset stanford_dogs downloaded and prepared to /root/tensorflow_datasets/stanford_dogs/0.2.0. Subsequent calls will reuse this data.


In [6]:
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [7]:
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

In [8]:
# One-hot / categorical encoding
def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

In [9]:
ds_test.filter

<bound method DatasetV2.filter of <_BatchDataset element_spec=(TensorSpec(shape=(64, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(64, 120), dtype=tf.float32, name=None))>>

In [46]:
import os
def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB3(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", name="pred")(x)

    # checkpoint_path = "/Users/wangshiyu/Desktop/CS 673/Term Project/cp.ckpt"
    # checkpoint_dir = os.path.dirname(checkpoint_path)

    # Create a callback that saves the model's weights
    # cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                    # save_weights_only=True,
                                                    # verbose=1)
    # Compile

    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [27]:
with strategy.scope():
    model = build_model(num_classes=NUM_CLASSES)

epochs = 50
# hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)
# plot_hist(hist)
checkpoint_path = "/content/checkpoints"  # 模型保存路径
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_best_only=True, save_weights_only=True, monitor="val_accuracy", mode="max", verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=100, monitor="val_accuracy", mode="max", restore_best_weights=True, verbose=1)

model.fit(ds_train, epochs=epochs, validation_data=ds_test, callbacks=[model_checkpoint, early_stopping], verbose=2)


43941136/43941136 [==============================] - 0s 0us/step
Epoch 1/50

Epoch 1: val_accuracy improved from -inf to 0.83092, saving model to /content/checkpoints
187/187 - 91s - loss: 1.7889 - accuracy: 0.7404 - val_loss: 0.7083 - val_accuracy: 0.8309 - 91s/epoch - 488ms/step
Epoch 2/50

Epoch 2: val_accuracy improved from 0.83092 to 0.84387, saving model to /content/checkpoints
187/187 - 68s - loss: 0.6423 - accuracy: 0.8745 - val_loss: 0.8825 - val_accuracy: 0.8439 - 68s/epoch - 363ms/step
Epoch 3/50

Epoch 3: val_accuracy did not improve from 0.84387
187/187 - 66s - loss: 0.4242 - accuracy: 0.9026 - val_loss: 1.0351 - val_accuracy: 0.8390 - 66s/epoch - 354ms/step
Epoch 4/50

Epoch 4: val_accuracy did not improve from 0.84387
187/187 - 66s - loss: 0.3401 - accuracy: 0.9180 - val_loss: 1.1398 - val_accuracy: 0.8347 - 66s/epoch - 354ms/step
Epoch 5/50

Epoch 5: val_accuracy did not improve from 0.84387
187/187 - 66s - loss: 0.3151 - accuracy: 0.9222 - val_loss: 1.0746 - val_accura

In [58]:
model.input_shape

(None, 224, 224, 3)

In [28]:
import tensorflow as tf


# save the model as SavedModel format
# tf.saved_model.save(model, '/content/efficentNetSavedModel')
# model.save('/content/efficientNet.h5')
model.save_weights('/content/model_weights.h5')

# Step 2 transfer TF model to onnx formate

In [30]:
! pip install onnxruntime
! pip install tf2onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.2/451.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 98.2 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.5.26
    Uninstalling flatbuffers-23.5.26:
      Successfully uninstalled flatbuffers-23.5.26


In [31]:
import tensorflow as tf
import tf2onnx
from tf2onnx.convert import from_keras
import onnx
# from tf2onnx.optimizer import optimize_graph
input_signature = [tf.TensorSpec([None, 224,224,3], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)
onnx.save(onnx_model, "/content/efficientNetB3.onnx")

In [ ]:
import onnx

def check_onnx_model(filename):
    try:
        onnx_model = onnx.load(filename)
        onnx.checker.check_model(onnx_model)
        print("onnxfile works.")
    except onnx.onnx_cpp2py_export.checker.ValidationError as e:
        print("onnxfile cannot work")
        print(e)

check_onnx_model("/content/efficientNetB3.onnx")


# Step3 Test onnx

In [44]:
import numpy as np
import onnxruntime as rt
from PIL import Image

# load onnx model
model_path = "/content/efficientNetB3.onnx"
session = rt.InferenceSession(model_path)

# preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.resize((224, 224))
    image_array = np.array(image.convert("RGB"))
    image_array = np.expand_dims(image_array, axis=0)
    image_array = image_array.astype(np.float32)
    return image_array

# prepare input data
input_name = session.get_inputs()[0].name
image_path = "/content/appenzeller.jpeg"
input_data = preprocess_image(image_path)

# get inference
outputs = session.run(None, {input_name: input_data})

# deal with the output
output_data = outputs[0]
output_data = np.squeeze(output_data)

In [45]:
# get the label index
predicted_class_index = np.argmax(output_data)
print("Predicted class index:", predicted_class_index)

# get the label name
class_labels = ds_info.features["label"].names  # 替换为你的类别标签
predicted_class_label = class_labels[predicted_class_index]
print("Predicted class label:", predicted_class_label)


Predicted class index: 89
Predicted class label: n02107908-appenzeller


In [38]:
ds_info.features["label"].names

['n02085620-chihuahua',
 'n02085782-japanese_spaniel',
 'n02085936-maltese_dog',
 'n02086079-pekinese',
 'n02086240-shih-tzu',
 'n02086646-blenheim_spaniel',
 'n02086910-papillon',
 'n02087046-toy_terrier',
 'n02087394-rhodesian_ridgeback',
 'n02088094-afghan_hound',
 'n02088238-basset',
 'n02088364-beagle',
 'n02088466-bloodhound',
 'n02088632-bluetick',
 'n02089078-black-and-tan_coonhound',
 'n02089867-walker_hound',
 'n02089973-english_foxhound',
 'n02090379-redbone',
 'n02090622-borzoi',
 'n02090721-irish_wolfhound',
 'n02091032-italian_greyhound',
 'n02091134-whippet',
 'n02091244-ibizan_hound',
 'n02091467-norwegian_elkhound',
 'n02091635-otterhound',
 'n02091831-saluki',
 'n02092002-scottish_deerhound',
 'n02092339-weimaraner',
 'n02093256-staffordshire_bullterrier',
 'n02093428-american_staffordshire_terrier',
 'n02093647-bedlington_terrier',
 'n02093754-border_terrier',
 'n02093859-kerry_blue_terrier',
 'n02093991-irish_terrier',
 'n02094114-norfolk_terrier',
 'n02094258-norwi